#### Creates yearly binary rasters for each climate criteria, where , if a grid cell meets the criteria for a given year it gets a 1 else it is assigned a zero.  C3 counts the number of days where minimum temp < -40 every year (not a binary raster). Years span Aug 1st ,year  to Aug. 1st, year +1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import rasterio
import fiona
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
import rasterio.mask
import datetime 
import gdal

In [2]:
def CreateRaster(lon,lat,data,resx,resy,save_directory):
    from osgeo import gdal
    from osgeo import gdal_array
    from osgeo import osr,ogr

    xmin,ymin,xmax,ymax = [lon.min(),lat.min(),lon.max(),lat.max()]
    nrows,ncols = np.shape(data)
    geotransform=(xmin-resy,resy,0,ymax+resx,0, -resx)  

    output_raster = gdal.GetDriverByName('GTiff').Create(save_directory,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file

    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(4326)                     # This one specifies WGS84 lat long.

    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system                                         
    output_raster.GetRasterBand(1).WriteArray(data)   # Writes my array to the raster
    output_raster.FlushCache()
    return

### C1

In [4]:
watersheds=['Spius']#,'Chilko','Chilcotin','Nation','Osilinka','Mesilinka','Stellako','Nautley']
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
vari='t2m'

GrowingSeason=pd.read_excel(path+'MPB_Constraints/GrowingSeason_WS.xlsx')

for WS in watersheds:
    
    #define the ending month and day of growing season 
    End_Month=GrowingSeason[GrowingSeason.WS==WS].End_Month.values[0]
    End_Day=GrowingSeason[GrowingSeason.WS==WS].End_Day.values[0]
    One=[];Two=[]
    
    #-----------------------------Growing Season degree day constraint-------------------------------------------
    for year in np.arange(1981,1982,1):
        Year_Array=[]
        
        #load average temperature data
        TB=pd.read_pickle(path+'Temp/Daily_Vars/%s_%s.pkl'%(vari,year))
        
        #loop through month and day until end of growing season
        for M in np.arange(8,End_Month+1,1):
            if M==End_Month:
                DAY=End_Day+1
            else:
                DAY=32
                
            for D in np.arange(1,DAY,1):
                working=TB[(TB.Month==M)&(TB.Day==D)]
                
                #check day and month combo exist
                if len(working)<1:
                    continue
                
                #calculating degree days for each day
                Var=working.Var.values[0]
                Shape=np.shape(Var)
                Num= np.empty([Shape[0],Shape[1]])
                for i in np.arange(0,Shape[0]):
                    for j in np.arange(0, Shape[1]):
                        
                        # if degree day is greater than 5.5 set array value to difference
                        if Var[i][j] >5.5:
                            Num[i][j]=Var[i][j]-5.5
                            
                        #otherwise set it to zero
                        else:
                            Num[i][j]=0
                Year_Array.append(Num)
         
        #sum degree days thoughout aug to end of growing season
        GrowingSeason_DD= np.sum(Year_Array,axis=0)  


        #------------------------------ Yearly Degree Day constraint----------------------------------------
        Year_Array=[]
        
        #open avererage temperature data from year and year+1
        TB=pd.read_pickle(path+'Temp/Daily_Vars/%s_%s.pkl'%(vari,year))
        TB2= pd.read_pickle(path+'Temp/Daily_Vars/%s_%s.pkl'%(vari,year+1))
        
        # determine degree day from Aug to end of year
        for M in np.arange(8,13,1):
            for D in np.arange(1,32,1):
                working=TB[(TB.Month==M)&(TB.Day==D)]
                
                #check day and month combo exist
                if len(working)<1:
                    continue
                
                #calculating degree days for each day
                Var=working.Var.values[0]
                Shape=np.shape(Var)
                Num= np.empty([Shape[0],Shape[1]])
                for i in np.arange(0,Shape[0]):
                    for j in np.arange(0, Shape[1]):
                        
                        # if degree day is greater than 5.5 set array value to difference
                        if Var[i][j] >5.5:
                            Num[i][j]=Var[i][j]-5.5
                            
                        #otherwise set it to zero    
                        else:
                            Num[i][j]=0
                Year_Array.append(Num)
                
        # determine degree days from Jan to Aug (year+1)
        for M in np.arange(1,8,1):
            for D in np.arange(1,32,1):
                working=TB2[(TB2.Month==M)&(TB2.Day==D)]
                
                #check day and month combo exist
                if len(working)<1:
                    continue
                    
                    
                #calculating degree days for each day
                Var=working.Var.values[0]
                Shape=np.shape(Var)
                Num= np.empty([Shape[0],Shape[1]])
                for i in np.arange(0,Shape[0]):
                    for j in np.arange(0, Shape[1]):
                        
                        # if degree day is greater than 5.5 set array value to difference
                        if Var[i][j] >5.5:
                            Num[i][j]=Var[i][j]-5.5
                            
                        #otherwise set it to zero    
                        else:
                            Num[i][j]=0
                Year_Array.append(Num)
                
        #calculate full year degree day total
        FullYear_DD= np.sum(Year_Array,axis=0)  


        Shape= np.shape(FullYear_DD)
        Binary_1=np.empty([Shape[0],Shape[1]])
        
        #------------------------------------ create binary array for C1---------------------------------
        
        
        for i in np.arange(0,Shape[0]):
            for j in np.arange(0,Shape[1]):
                
                #if both growing season and year degree day constraints are  met --> Ture 
                if (FullYear_DD[i][j]>833)&(GrowingSeason_DD[i][j]>305):
                    Binary_1[i][j]=1
                    
                # if both are not met --> False   
                else:
                    Binary_1[i][j]=0
                    
        #Save binary array for C1 
        np.save(path+ 'MPB_Constraints/P1/%s_%s'%(year,WS),FullYear_DD)

### C2

In [3]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
vari='t2m'

for year in np.arange(1981,1982,1):
    Year_Array=[]
    
    #open minimum temperature arrays for year and year+1 
    TB=pd.read_pickle(path+'Temp/Daily_Vars/%s_%s_MIN.pkl'%(vari,year))
    TB2= pd.read_pickle(path+'Temp/Daily_Vars/%s_%s_MIN.pkl'%(vari,year+1))
    
    # loops through Nov and Dec of year
    for M in np.arange(11,13,1):
        for D in np.arange(1,32,1):
            working=TB[(TB.Month==M)&(TB.Day==D)]
            
            # make sure month day combo exists 
            if len(working)<1:
                continue
            
            #create binary array for each day: min temps below -40 and days above
            Var=working.Var.values[0]
            Shape=np.shape(Var)
            Num= np.empty([Shape[0],Shape[1]])
            for i in np.arange(0,Shape[0]):
                for j in np.arange(0, Shape[1]):
                    if Var[i][j] <-40:
                        Num[i][j]=1
                    else:
                        Num[i][j]=0
            Year_Array.append(Num)
    
    #loops through Jan through May of year+1
    for M in np.arange(1,5,1):
        for D in np.arange(1,32,1):
            working=TB2[(TB2.Month==M)&(TB2.Day==D)]
            
            # make sure month day combo exists 
            if len(working)<1:
                continue
            
            #create binary array for each day: min temps below -40 and days above
            Var=working.Var.values[0]
            Shape=np.shape(Var)
            Num= np.empty([Shape[0],Shape[1]])
            for i in np.arange(0,Shape[0]):
                for j in np.arange(0, Shape[1]):
                    if Var[i][j] <-40:
                        Num[i][j]=1
                    else:
                        Num[i][j]=0
            Year_Array.append(Num)
            
    # sums all binary arrays together --> value in each grid cell tells how many days below -40 occurred that year        
    Cold_Days= np.sum(Year_Array,axis=0) 
    
    # save C2 constraint by year
    np.save(path+ 'MPB_Constraints/P2/%s'%year,Cold_Days)

### C3

In [ ]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
vari='t2m'

for year in np.arange(1981,1982,1):
    Year_Array=[]
    
    #load maximum daily temperature data
    TB=pd.read_pickle(path+'Temp/Daily_Vars/%s_%s_MAX.pkl'%(vari,year))
    Var=[]
    
    # Select Aug for analysis
    for M in np.arange(8,9,1):
        for D in np.arange(1,32,1):
            working= TB[(TB.Month==M)&(TB.Day==D)]
            
            # make sure month day combo exists 
            if len(working)<1:
                continue

            Var.append(working.Var.values[0])
    
    # Find the mean maximum august temperatures
    Max_Avg_Aug=np.mean(Var, axis=0)
    
    #create binary array for each day: max temps above 18.3 (true) and those below 18.3 (false)
    Shape=np.shape(Max_Avg_Aug)
    Num= np.empty([Shape[0],Shape[1]])
    for i in np.arange(0,Shape[0]):
        for j in np.arange(0,Shape[1]):
            if Max_Avg_Aug[i][j]>18.3:
                Num[i][j]=1
            else:
                Num[i][j]=0
                
    #save C2 constraint by year
    np.save(path+ 'MPB_Constraints/P3/%s'%year,Num)

### C4

In [4]:
path='D:/UNBC/HydroMet_Project/Data/ERA5_land/'
vari='tp'

Tot_Precip=[]
for year in np.arange(1981,1982,1):

    #open total precipitation arrays for year and year+1 
    TB2=pd.read_pickle(path+'Tot_Precip/Daily_Vars/%s_%s.pkl'%(vari,year+1))
    Var=[]

    #select months April, May, and June of year +1
    for M in np.arange(4,7,1):
        for D in np.arange(1,32,1):
            working= TB2[(TB2.Month==M)&(TB2.Day==D)]
            
            #make sure month and day combo exists
            if len(working)<1:
                continue

            Var.append(working.Var.values[0])
            
    # total spring precipitation      
    Spring_Precip=np.sum(Var, axis=0)
    Tot_Precip.append(Spring_Precip)

#average total precipitation 
T= np.mean(Tot_Precip,axis=0)

Shape=np.shape(Tot_Precip)


# create binary array for each year: if precipitation is less than average (True) or 
#                    if precipitation is greater than average (False)
Year=1981
for N in np.arange(0,Shape[0]):
    Num= np.empty([Shape[1],Shape[2]])
    working=Tot_Precip[N]
    for i in np.arange(0,Shape[1]):
        for j in np.arange(0,Shape[2]):
            if working[i][j]<T[i][j]:
                Num[i][j]=1
            else:
                Num[i][j]=0
    
    
    np.save(path+ 'MPB_Constraints/P4/%s'%Year,Num)
    Year=Year+1